# Сбор данных с сайта РИА Новости

*Ноутбук сделан по мотивам ноутбука авторства Марии Гольдштейн и Кирилла Долматова.*

Установим и импортируем необходимые библиотеки.

In [1]:
%%capture
!pip install selenium

In [2]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

Далее зададим необходимые константы.

`DEPTH` - число страниц для скрапинга
 -'Общество/Россия' : 0 +
- 'Экономика' : 1 +
- 'Силовые структуры' : 2 +
- 'Бывший СССР' : 3
- 'Спорт' : 4
- 'Забота о себе' : 5
- 'Строительство' : 6
- 'Туризм/Путешествия' : 7
- 'Наука и техника' : 8

In [12]:
SLEEP = 1.25
DEPTH = 100
BASE_URL = "https://ria.ru/"
TOPICS = ["society", "economy", "defense_safety", "culture", "tourism", "science"]

In [4]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None

In [81]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

Посмотрим на функцию для скачивания страниц.

In [10]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

            # push to list 20 next articles
            driver.execute_script(
                "document.getElementsByClassName('list-more')[0].click()"
            )
            time.sleep(1)

            # scroll page to automatically load more articles
            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(
                        f"window.scrollTo(0, document.body.scrollHeight - 1200)"
                    )
                    time.sleep(1)
                except:
                    pass

            # find all pages
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"class": "list", "itemtype": "http://schema.org/ItemList"}
            )
            items += scope.find_all("div", {"class": "list-item"})

            # number of pages can not be multiple of deepth*20
            # that's why we count topics_order dynamically
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))
        except:
            pass

    return items, topics_order

Посмотрим на функцию для парсинга скачанных страниц.

In [11]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find("a", {"class": "list-item__image"})["href"]

    # article id
    s = re.findall(r"\d+.html", article.url)[0]
    article.id = s[: s.find(".")]

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article source
    source = article.url[8 : article.url.find(".")]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find(
        "div",
        {
            "class": lambda x: x and (x.find(f"article m-article m-{source}") > -1),
            "data-article-id": article.id,
        },
    )

    if not obj:
        obj = soup.find(
            "div",
            {
                "class": lambda x: x and (x.find(f"article m-video m-{source}") > -1),
                "data-article-id": article.id,
            },
        )

    # process article title
    title = obj.find("div", {"class": "article__title"})
    title_2 = obj.find("h1", {"class": "article__title"})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ""

    # article subtitle
    subtitle = obj.find("h1", {"class": "article__second-title"})
    article.subtitle = subtitle.text if subtitle else ""

    # article content
    article.content = obj.find(
        "div", {"class": "article__body js-mediator-article mia-analytics"}
    ).text

    # article datetime
    article.datetime = obj.find("div", {"class": "article__info-date"}).find("a").text

    # article number of views
    # article.views = int(obj.find('span', {'class': 'statistic__item m-views'}).text)

    return article

Скачаем новости (количество регулируется константой DEPTH выше).

In [13]:
# get pages and topics
pages, topics_order = get_pages()

100%|██████████| 6/6 [10:53<00:00, 108.87s/it]   


Сохраним результат в файл.

In [14]:
with open("ria_pages.txt", "w", encoding="utf-8") as f:
    for page in pages:
        f.write("%s\n" % page)

## Вопрос 1

Сколько страниц `pages` мы спарсили?

In [ ]:
# ваш код здесь

Распарсим каждую страницу и достанем из нее необходимые атрибуты:

* в цикле примените функцию `parse_page` к каждой странице и сохраните результат в объект `res`

* затем добавьте `res` в список `data`


Для скорости распарсим только первые 100 страниц.

In [79]:
data, topics_order_fixed = [], []

In [82]:

for num, page in enumerate(tqdm(pages[2430:])):
    try:
        res = parse_page(page)
        data.append(res)
        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

100%|██████████| 5998/5998 [42:09<00:00,  2.37it/s]  


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.265)
Stacktrace:
0   chromedriver                        0x0000000104bce138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000104bc6988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x000000010462c968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000104607e44 core::str::slice_error_fail::ha0e52dbcb60e6bae + 3780
4   chromedriver                        0x0000000104696c84 cxxbridge1$string$len + 524200
5   chromedriver                        0x000000010469c60c cxxbridge1$string$len + 547120
6   chromedriver                        0x0000000104665564 cxxbridge1$string$len + 321672
7   chromedriver                        0x00000001046661b4 cxxbridge1$string$len + 324824
8   chromedriver                        0x0000000104b98fc0 cxxbridge1$str$ptr + 3436424
9   ch

Преобразуем в датафрейм полученные данные.

## Вопрос 2

Сколько столбцов в таблице `df`?

In [75]:
df = pd.DataFrame(data=data)


In [77]:
len(data)

0

In [78]:
df

""


In [55]:
df.to_pickle('2430.pkl')

In [56]:
df_loaded = pd.read_pickle('2430.pkl')
df_loaded

,id,url,title,subtitle,content,datetime
0,1993321188,https://ria.ru/20250112/videoigry-1993321188.html,Володин ответил на сообщения о возможном запре...,"Володин: нужен не запрет видеоигр, а защита де...","МОСКВА, 12 янв — РИА Новости. Председатель Гос...",09:59 12.01.2025
1,1993318917,https://ria.ru/20250112/tigr-1993318917.html,Эксперт прокомментировал фото девушки с амурск...,"Глава центра ""Амурский тигр"" назвал фото девуш...","ВЛАДИВОСТОК, 12 янв — РИА Новости. Фотографии,...",09:40 12.01.2025
2,1993317726,https://ria.ru/20250112/pulkovo-1993317726.html,В Пулково приостановили прием и отправку возду...,В Пулково приостановили полеты воздушных судов...,"МОСКВА, 12 янв - РИА Новости. Петербургский аэ...",09:22 12.01.2025
3,1993111587,https://ria.ru/20250112/prazdnik-1993111587.html,"12 января: какой сегодня праздник, что отмечаю...",,ОглавлениеКакой сегодня праздник в РоссииПрофе...,06:00 12.01.2025
4,1993305395,https://ria.ru/20250112/vrach-1993305395.html,"Ортопед рассказал, к чему приводит сидячий обр...",Ортопед Сорокин: сидячий образ жизни приводит ...,"МОСКВА, 12 янв — РИА Новости. Сидячий образ жи...",03:12 12.01.2025
...,...,...,...,...,...,...
2405,1993247337,https://ria.ru/20250111/nozh-1993247337.html,Николаев подарил якутскому штурмовику новый нож,"Глава Якутии подарил нож российскому бойцу, од...","МОСКВА, 11 янв — РИА Новости. Глава Якутии Айс...",12:04 11.01.2025
2406,1993245630,https://ria.ru/20250111/akhmat-1993245630.html,"В ""Ахмате"" рассказали о сражении группы РЭБ с ...","Командир ""Пресс"": группа РЭБ ""Ахмата"" в Курско...","КУРСКАЯ ОБЛАСТЬ, 11 янв - РИА Новости. Группа ...",11:46 11.01.2025
2407,1993242276,https://ria.ru/20250111/aviabomby-1993242276.html,Рогов рассказал об ударах по позициям ВСУ в Дн...,Рогов: ВС РФ бьют корректируемыми авиабомбами ...,"СИМФЕРОПОЛЬ, 11 янв – РИА Новости. Российские ...",11:13 11.01.2025
2408,1993241352,https://ria.ru/20250111/udary-1993241352.html,Авиация нанесла удар по ВСУ в Запорожской обла...,Рогов: авиация ночью ударила ФАБ-500 по скопле...,"СИМФЕРОПОЛЬ, 11 янв – РИА Новости. Российская ...",11:02 11.01.2025


Добавьте в таблицу `df` столбец `topic`, содержащий список `topics_order_fixed`

In [ ]:
df["topic"] = # ваш код здесь

## Вопрос 3

Какой `topic` имеет новость из первой строки таблицы?

In [ ]:
# ваш код здесь

Сохраним скачанные данные в файл.

In [ ]:
df.to_pickle("df_ria.p", compression="gzip")

## Вопрос 4

Есть ли пропуски в каких-либо столбцах в таблице `df`?

In [ ]:
# ваш код здесь